In [1]:
# The project aims to classify the customers based on recency (Now - last purchase date), frequency (times of purchase) and 
# monetary (money spent each invoice) score. Then rank customers into 4 categories to determine their loyality level 
# The marketing team can then do target marketing for each customer, which maxmise outcome of the contrained budget

In [2]:
#import liabiraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [3]:
# Read the data as dataframe, also drop all null value in rows for demonstration purpose. Please note should always 
# discuss with subject matter expert (SME) regarding why null exists in practice, and see how to deal with null value
df = pd.read_csv("online_shoppers.csv",encoding = "ISO-8859-1").dropna()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
# Noted InvoiceDate is object, will need to transform its type later
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [5]:
# Creating Sales column for monetary 
df['sales'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [6]:
# Dates - Transform Date variable, and get the last date
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#Get last date available
last_date = df['InvoiceDate'].max() + timedelta(days = 1)
last_date

Timestamp('2011-12-10 12:50:00')

In [7]:
# Aggregate on Customer Level, get the most recent purchase date till now, Number of invoices and 
# total sales from each customer
df = df.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (last_date - x.max()).days,
    'InvoiceNo': 'count',
    'sales': 'sum'
})
df.head()

,InvoiceDate,InvoiceNo,sales
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [8]:
# Create Monetary column, get the sales per invoice
df['monetary'] = df.sales / df.InvoiceNo
df.head()

,InvoiceDate,InvoiceNo,sales,monetary
CustomerID,,,,
12346.0,326,2,0.00,0.000000
12347.0,2,182,4310.00,23.681319
12348.0,75,31,1797.24,57.975484
12349.0,19,73,1757.55,24.076027
12350.0,310,17,334.40,19.670588


In [9]:
# Dropping Sales column as not required
df = df.drop(columns = "sales")
df.head()

,InvoiceDate,InvoiceNo,monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484
12349.0,19,73,24.076027
12350.0,310,17,19.670588


In [10]:
# Changing columns' names
df.rename(columns = {'InvoiceDate': 'Recency',
                     'InvoiceNo': 'Frequency'}, inplace = True)
df.head()

,Recency,Frequency,monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484
12349.0,19,73,24.076027
12350.0,310,17,19.670588


In [11]:
# Create scale for recency, frequency and monetary, using 4 scale
df['F'] = pd.qcut(x = df['Frequency'], q = 4, labels = range(1, 5, 1))
df['M'] = pd.qcut(x = df['monetary'], q = 4, labels = range(1, 5, 1))
df['R'] = pd.qcut(x = df['Recency'], q = 4, labels = range(4, 0, -1))
df.head()

,Recency,Frequency,monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2
12349.0,19,73,24.076027,3,4,3
12350.0,310,17,19.670588,1,3,1


In [12]:
# Sum the RFM scales
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head()

,Recency,Frequency,monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12
12348.0,75,31,57.975484,2,4,2,8
12349.0,19,73,24.076027,3,4,3,10
12350.0,310,17,19.670588,1,3,1,5


In [13]:
# Create the RFM function based on total RFM score, rank customers into 4 royality levels
def rfm_segment(df):
  if df['RFM'] >= 11:
    return 'Gold level'
  elif ((df['RFM'] >=8) and (df['RFM'] < 11)):
    return 'Silver level'
  elif ((df['RFM'] >=6) and (df['RFM'] < 8)):
    return 'Above average'
  else:
    return 'Average & Below'

In [14]:
# Apply RFM function,we can then export the file to marketing team. Marketing team can design the campaign for cusotmers 
# based on their RFM level. For instance, fewer discount to Gold & Silver levels and higher discount for Above/ AVG 
# and below customers
df['RFM_level'] = df.apply(rfm_segment, axis = 1)
df.tail()

,Recency,Frequency,monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
18280.0,278,10,18.060000,1,3,1,5,Average & Below
18281.0,181,7,11.545714,1,2,1,4,Average & Below
18282.0,8,13,13.584615,1,2,4,7,Above average
18283.0,4,756,2.771005,4,1,4,9,Silver level
18287.0,43,70,26.246857,3,4,3,10,Silver level


In [15]:
# Looking into the segments, and noted 1. silver & above AVG are major cusotmers and 2 .average & below customers has 
# negative mean for monetary, which is unusual may need to find out more with SME
df.groupby('RFM_level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'monetary': ['mean', 'count']
})

Recency   Frequency   monetary      
                       mean        mean       mean count
RFM_level                                               
Above average    118.708869   39.213947  38.033024  1477
Average & Below  221.483400   22.811421  -0.586430   753
Gold level         9.973262  244.852941  45.483949   374
Silver level      32.007919  135.835973  30.169670  1768